In [ ]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge,EdgeOptions

options=EdgeOptions()
options.use_chromium=True
driver =Edge(options=options)

driver.get('https://twitter.com/login')

def get_username(user):
    username = driver.find_element_by_xpath('//input[@name="username"]')
    username.send_keys(user)
    username.send_keys(Keys.RETURN)
    
def get_phone(ph):
    phone = driver.find_element_by_xpath('//input[@name="text"]')
    phone.send_keys(ph)
    phone.send_keys(Keys.RETURN)

def get_password(password):
    passfield = driver.find_element_by_xpath('//input[@name="password"]')
    passfield.send_keys(password)
    passfield.send_keys(Keys.RETURN)

def search(searchterm):
    search_input = driver.find_element_by_xpath('//input[@class="r-30o5oe r-1niwhzg r-17gur6a r-1yadl64 r-deolkf r-homxoj r-poiln3 r-7cikom r-1ny4l3l r-xyw6el r-641cr4 r-1dz5y72 r-fdjqy7 r-13qz1uu"]')
    search_input.send_keys(searchterm)
    search_input.send_keys(Keys.RETURN)

def open_detail():
    driver.find_element_by_link_text('Latest').click()
    
def get_tweet_data(card):
    """Extracting data from tweet data""" 
    username= card.find_element_by_xpath('.//span').text
    handle= card.find_element_by_xpath('.//span[contains(text(),"@")]').text
    try:
        postdate=card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    text = card.find_element_by_xpath('.//div[@class="css-1dbjc4n"]').text
    reply_count=card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    try:
        retweet_count = card.find_element_by_xpath('.//div[data-testid="retweet"]').text
    except:
        retweet_count='0'
    try:
        like_count = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    except:
        like_count='0'
    
    tweet=(username,handle,postdate,text,reply_count,retweet_count,like_count)
    print(tweet)
    return tweet

def save_in_csv(datas):
    with open('Twitter Squeaked.csv','w',newline='',encoding='utef-8') as f:
        header = ['User Name','Handle','TimeStamp','Comments','Like','Retweets','Text']
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(datas)
def start_scraping():
    data=[]
    tweet_ids =set()
    last_position= driver.execute_script("return window,pageYOffset;")
    scrolling =True
    
    while scrolling:
        page_cards =driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
        for card in page_cards:
            tweet = get_tweet_data(card)
            if tweet:
                tweet_id=''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    data.append(tweet)
    
    scroll_attempt=0
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position ==curr_position:
            scroll_attempt+=1
            
            if scroll_attempt>= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr-position
            break
    save_in_csv(data)
    print(data)


if __name__=='__main__':
        
    u = input('username:')    
    done=input('Done?')
    get_username(u)
    n = input('Phone/Username:')
    done=input('Done?')
    get_phone(n)
    p = getpass('password:')
    done=input('Done?')
    get_password(p)
    si = input("Search: ")
    done=input('Done?')
    search(si)
    done=input('Done?')
    open_detail()
    done=input('Done?')
    start_scraping()


username:laibahazhar@gmail.com
Done?
Phone/Username:LaibahAzhar
Done?
password:········
Done?
Search: Dharna
Done?
Done?
Done?
('Karachi Ki Awaz ®', '@Karachi_Update', '2021-10-19T17:38:29.000Z', 'Karachi Ki Awaz ®\n@Karachi_Update\n·\n1m\n#Lahore:\n#TLP Ne Dharna Day Dia, Zalim Hukomat K Khilaf Awam Ka Samandar Sarkon Pr Aagaya Hai, #Pakistani Media Aisi News Nahi Chalata Jo Such Hai.\n0:27\n1\n2', '', '0', '2')
('Farhan Mallick', '@FGMallick', '2021-10-19T17:34:31.000Z', 'Farhan Mallick\n@FGMallick\n·\n5m\n#TLP converts its protest into a formal road blocking Dharna in Lahore. House of Cards episode #4269619\nThe following media includes potentially sensitive content. Change settings\nView\n2', '', '0', '2')
('Ali_', '@RajaPaki1', '2021-10-19T17:32:52.000Z', 'Ali_\n@RajaPaki1\n·\n7m\nImran Khan ki looli langri Hakumat khatam krne k liye 3 mahaaz khul chukay hain\n\n1 Balochistan me Adam aitmad ki tehreek\n2 PDM ki Mehngai k khilaaf tehreek\n3 TLP ka dharna', '', '0', '')
('Bhartiya D

('Ali_', '@RajaPaki1', '2021-10-19T17:32:52.000Z', 'Ali_\n@RajaPaki1\n·\n7m\nImran Khan ki looli langri Hakumat khatam krne k liye 3 mahaaz khul chukay hain\n\n1 Balochistan me Adam aitmad ki tehreek\n2 PDM ki Mehngai k khilaaf tehreek\n3 TLP ka dharna', '', '0', '')
('Bhartiya Dharna Party', '@SBKA7SbseBAKWAS', '2021-10-19T17:31:39.000Z', 'Bhartiya Dharna Party\n@SBKA7SbseBAKWAS\n·\n8m\nReplying to \n@RealWBTruth\nYes, She is against Tista Water Sharing', '', '0', '')
('Saad Bin Azam Hashmi', '@saadhashmi10', '2021-10-19T17:29:42.000Z', 'Saad Bin Azam Hashmi\n@saadhashmi10\n·\n10m\nRoad closed from Yateem khana to scheme morr due to dharna, use alternative routes.\n#TLP\n2', '', '0', '2')
('Bhartiya Dharna Party', '@SBKA7SbseBAKWAS', '2021-10-19T17:29:03.000Z', 'Bhartiya Dharna Party\n@SBKA7SbseBAKWAS\n·\n10m\nIsnt is funny, BJYM Bengal Leaders are tagging \n@BAPS\n to speak on #BangladeshiHindusInDanger \n\nThere own Leader Modi , Shah, Rajnath are silent\n4\n6', '', '0', '6')
('Kira

('Bhartiya Dharna Party', '@SBKA7SbseBAKWAS', '2021-10-19T17:29:03.000Z', 'Bhartiya Dharna Party\n@SBKA7SbseBAKWAS\n·\n10m\nIsnt is funny, BJYM Bengal Leaders are tagging \n@BAPS\n to speak on #BangladeshiHindusInDanger \n\nThere own Leader Modi , Shah, Rajnath are silent\n4\n6', '', '0', '6')
('Kiran Butt', '@qiranbutt', '2021-10-19T17:25:36.000Z', "Kiran Butt\n@qiranbutt\n·\n14m\n#Breaking Footage coming out from #TLP's dharna right now. \n@IGP_Punjab\n @GovtofPunjabPK\n @OfficialDPRPP\n @UsmanAKBuzdar\n anyone has a plan how to deal with the protesters?\n2\n3", '2', '0', '3')
('Karachi Ki Awaz ®', '@Karachi_Update', '2021-10-19T17:38:29.000Z', 'Karachi Ki Awaz ®\n@Karachi_Update\n·\n1m\n#Lahore:\n#TLP Ne Dharna Day Dia, Zalim Hukomat K Khilaf Awam Ka Samandar Sarkon Pr Aagaya Hai, #Pakistani Media Aisi News Nahi Chalata Jo Such Hai.\n0:03\n1\n2', '', '0', '2')
('Farhan Mallick', '@FGMallick', '2021-10-19T17:34:31.000Z', 'Farhan Mallick\n@FGMallick\n·\n5m\n#TLP converts its protest i

('Farhan Mallick', '@FGMallick', '2021-10-19T17:34:31.000Z', 'Farhan Mallick\n@FGMallick\n·\n5m\n#TLP converts its protest into a formal road blocking Dharna in Lahore. House of Cards episode #4269619\nThe following media includes potentially sensitive content. Change settings\nView\n2', '', '0', '2')
('Ali_', '@RajaPaki1', '2021-10-19T17:32:52.000Z', 'Ali_\n@RajaPaki1\n·\n7m\nImran Khan ki looli langri Hakumat khatam krne k liye 3 mahaaz khul chukay hain\n\n1 Balochistan me Adam aitmad ki tehreek\n2 PDM ki Mehngai k khilaaf tehreek\n3 TLP ka dharna', '', '0', '')
('Bhartiya Dharna Party', '@SBKA7SbseBAKWAS', '2021-10-19T17:31:39.000Z', 'Bhartiya Dharna Party\n@SBKA7SbseBAKWAS\n·\n8m\nReplying to \n@RealWBTruth\nYes, She is against Tista Water Sharing', '', '0', '')
('Saad Bin Azam Hashmi', '@saadhashmi10', '2021-10-19T17:29:42.000Z', 'Saad Bin Azam Hashmi\n@saadhashmi10\n·\n10m\nRoad closed from Yateem khana to scheme morr due to dharna, use alternative routes.\n#TLP\n2', '', '0', '2'

('Saad Bin Azam Hashmi', '@saadhashmi10', '2021-10-19T17:29:42.000Z', 'Saad Bin Azam Hashmi\n@saadhashmi10\n·\n10m\nRoad closed from Yateem khana to scheme morr due to dharna, use alternative routes.\n#TLP\n2', '', '0', '2')
('Bhartiya Dharna Party', '@SBKA7SbseBAKWAS', '2021-10-19T17:29:03.000Z', 'Bhartiya Dharna Party\n@SBKA7SbseBAKWAS\n·\n10m\nIsnt is funny, BJYM Bengal Leaders are tagging \n@BAPS\n to speak on #BangladeshiHindusInDanger \n\nThere own Leader Modi , Shah, Rajnath are silent\n5\n6', '', '0', '6')
('Kiran Butt', '@qiranbutt', '2021-10-19T17:25:36.000Z', "Kiran Butt\n@qiranbutt\n·\n14m\n#Breaking Footage coming out from #TLP's dharna right now. \n@IGP_Punjab\n @GovtofPunjabPK\n @OfficialDPRPP\n @UsmanAKBuzdar\n anyone has a plan how to deal with the protesters?\n2\n3", '2', '0', '3')
('Karachi Ki Awaz ®', '@Karachi_Update', '2021-10-19T17:38:29.000Z', 'Karachi Ki Awaz ®\n@Karachi_Update\n·\n1m\n#Lahore:\n#TLP Ne Dharna Day Dia, Zalim Hukomat K Khilaf Awam Ka Samandar Sa

('Karachi Ki Awaz ®', '@Karachi_Update', '2021-10-19T17:38:29.000Z', 'Karachi Ki Awaz ®\n@Karachi_Update\n·\n1m\n#Lahore:\n#TLP Ne Dharna Day Dia, Zalim Hukomat K Khilaf Awam Ka Samandar Sarkon Pr Aagaya Hai, #Pakistani Media Aisi News Nahi Chalata Jo Such Hai.\n0:24\n1\n2', '', '0', '2')
('Farhan Mallick', '@FGMallick', '2021-10-19T17:34:31.000Z', 'Farhan Mallick\n@FGMallick\n·\n5m\n#TLP converts its protest into a formal road blocking Dharna in Lahore. House of Cards episode #4269619\nThe following media includes potentially sensitive content. Change settings\nView\n2', '', '0', '2')
('Ali_', '@RajaPaki1', '2021-10-19T17:32:52.000Z', 'Ali_\n@RajaPaki1\n·\n7m\nImran Khan ki looli langri Hakumat khatam krne k liye 3 mahaaz khul chukay hain\n\n1 Balochistan me Adam aitmad ki tehreek\n2 PDM ki Mehngai k khilaaf tehreek\n3 TLP ka dharna', '', '0', '')
('Bhartiya Dharna Party', '@SBKA7SbseBAKWAS', '2021-10-19T17:31:39.000Z', 'Bhartiya Dharna Party\n@SBKA7SbseBAKWAS\n·\n8m\nReplying to \n@R

('Bhartiya Dharna Party', '@SBKA7SbseBAKWAS', '2021-10-19T17:31:39.000Z', 'Bhartiya Dharna Party\n@SBKA7SbseBAKWAS\n·\n8m\nReplying to \n@RealWBTruth\nYes, She is against Tista Water Sharing', '', '0', '')
('Saad Bin Azam Hashmi', '@saadhashmi10', '2021-10-19T17:29:42.000Z', 'Saad Bin Azam Hashmi\n@saadhashmi10\n·\n10m\nRoad closed from Yateem khana to scheme morr due to dharna, use alternative routes.\n#TLP\n2', '', '0', '2')
('Bhartiya Dharna Party', '@SBKA7SbseBAKWAS', '2021-10-19T17:29:03.000Z', 'Bhartiya Dharna Party\n@SBKA7SbseBAKWAS\n·\n10m\nIsnt is funny, BJYM Bengal Leaders are tagging \n@BAPS\n to speak on #BangladeshiHindusInDanger \n\nThere own Leader Modi , Shah, Rajnath are silent\n5\n6', '', '0', '6')
('Kiran Butt', '@qiranbutt', '2021-10-19T17:25:36.000Z', "Kiran Butt\n@qiranbutt\n·\n14m\n#Breaking Footage coming out from #TLP's dharna right now. \n@IGP_Punjab\n @GovtofPunjabPK\n @OfficialDPRPP\n @UsmanAKBuzdar\n anyone has a plan how to deal with the protesters?\n2\n3"

('Kiran Butt', '@qiranbutt', '2021-10-19T17:25:36.000Z', "Kiran Butt\n@qiranbutt\n·\n14m\n#Breaking Footage coming out from #TLP's dharna right now. \n@IGP_Punjab\n @GovtofPunjabPK\n @OfficialDPRPP\n @UsmanAKBuzdar\n anyone has a plan how to deal with the protesters?\n2\n3", '2', '0', '3')
('Karachi Ki Awaz ®', '@Karachi_Update', '2021-10-19T17:38:29.000Z', 'Karachi Ki Awaz ®\n@Karachi_Update\n·\n1m\n#Lahore:\n#TLP Ne Dharna Day Dia, Zalim Hukomat K Khilaf Awam Ka Samandar Sarkon Pr Aagaya Hai, #Pakistani Media Aisi News Nahi Chalata Jo Such Hai.\n0:19\n1\n1\n2', '1', '0', '2')
('Farhan Mallick', '@FGMallick', '2021-10-19T17:34:31.000Z', 'Farhan Mallick\n@FGMallick\n·\n5m\n#TLP converts its protest into a formal road blocking Dharna in Lahore. House of Cards episode #4269619\nThe following media includes potentially sensitive content. Change settings\nView\n2', '', '0', '2')
('Ali_', '@RajaPaki1', '2021-10-19T17:32:52.000Z', 'Ali_\n@RajaPaki1\n·\n7m\nImran Khan ki looli langri Hakumat 